In [1]:
%cd ..

/home/oscarn/flan-gpt2


In [2]:
import torch
from model_utils import GPT2Model
from data_utils import create_instruct_dataset
from torch.utils.data import DataLoader

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
checkpoint = 'openai-community/gpt2'

cuda


In [4]:
gpt2_model = GPT2Model(checkpoint, device)

# Load model and tokenizer
model = gpt2_model.get_model()
tokenizer = gpt2_model.get_tokenizer()

In [5]:
# Load instruct dataset (4 tasks)
datasets_names = ["common_gen"]
dataset = create_instruct_dataset(datasets_names)

# Tokenize examples
tokenized_dataset = dataset.map(
    gpt2_model.tokenize_function,
    remove_columns=["prompt", "completion"],
    batched=True,
    batch_size=500
)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [6]:
example = tokenizer.decode(tokenized_dataset["input_ids"][0])
print(example)

Generate a sentence, and then tell me the concepts included in that sentence. ### Response: Sentence:
The people are on mountain snowboarding down the slope.

Concepts:
['slope','snowboard','mountain']


In [7]:
train_dataloader = DataLoader(
    tokenized_dataset, shuffle=False, batch_size=1, collate_fn=gpt2_model.get_collator()
)

In [8]:
first_batch1 = next(iter(train_dataloader))
example = tokenizer.decode(first_batch1["input_ids"][0])
print(example)

Generate a sentence, and then tell me the concepts included in that sentence. ### Response: Sentence:
The people are on mountain snowboarding down the slope.

Concepts:
['slope','snowboard','mountain']


/home/oscarn/miniconda3/envs/transformers-311/lib/python3.11/site-packages/trl/trainer/utils.py:160: UserWarning: Could not find response key ` ### Response: ` in the following instance: Generate a sentence, and then tell me the concepts included in that sentence. ### Response: Sentence:
The people are on mountain snowboarding down the slope.

Concepts:
['slope','snowboard','mountain'] This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnings.warn(


In [9]:
from trl import DataCollatorForCompletionOnlyLM

In [10]:
sft_dataloader = DataLoader(
    tokenized_dataset, shuffle=False, batch_size=1, collate_fn=DataCollatorForCompletionOnlyLM(tokenizer=tokenizer,response_template=[1593])
)

In [11]:
first_batch2 = next(iter(sft_dataloader))
example = tokenizer.decode(first_batch2["input_ids"][0])
print(example)

Generate a sentence, and then tell me the concepts included in that sentence. ### Response: Sentence:
The people are on mountain snowboarding down the slope.

Concepts:
['slope','snowboard','mountain']


/home/oscarn/miniconda3/envs/transformers-311/lib/python3.11/site-packages/trl/trainer/utils.py:160: UserWarning: Could not find response key `[1593]` in the following instance: Generate a sentence, and then tell me the concepts included in that sentence. ### Response: Sentence:
The people are on mountain snowboarding down the slope.

Concepts:
['slope','snowboard','mountain'] This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnings.warn(


In [12]:
old_labels = first_batch2["labels"][0]

In [13]:
old_labels

tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100])

In [18]:
new_labels = old_labels[old_labels != -100]
example = tokenizer.decode(new_labels)
print(example)

In [19]:
print(tokenized_dataset["input_ids"][0])

[3103, 984, 82, 25, 37250, 67, 680, 3256, 705, 2655, 303, 3256, 705, 2118, 2899, 415, 20520, 198, 198, 16594, 257, 6827, 326, 3407, 477, 777, 2456, 13, 67, 680, 4983, 379, 262, 7072]


In [20]:
print(tokenized_dataset["targets"][0])

[67, 680, 4983, 379, 262, 7072]


In [21]:
tokenizer.encode(" ### Response:")

[44386, 18261, 25]

In [28]:
tokenizer.encode("Hello ### Response: [dsad")

[15496, 44386, 18261, 25, 685, 9310, 324]

In [22]:
tokenizer.decode([50256])

'<|endoftext|>'

In [114]:
special_tokens_dict = {"pad_token": "<|pad|>"}
tokenizer.add_special_tokens(special_tokens_dict)

1

In [1]:
%cd ..

/home/oscarn/flan-gpt2


/home/oscarn/miniconda3/envs/transformers-311/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from model_utils import GPT2Model
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
checkpoint = 'OscarNav/gpt2-multitask-4_V2'

cuda


In [3]:
gpt2_model = GPT2Model(checkpoint, device)

config.json:   0%|          | 0.00/984 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

In [4]:
model = gpt2_model.get_model()

In [6]:
model.push_to_hub?

Signature:
model.push_to_hub(
    repo_id: str,
    use_temp_dir: Optional[bool] = None,
    commit_message: Optional[str] = None,
    private: Optional[bool] = None,
    token: Union[bool, str, NoneType] = None,
    max_shard_size: Union[int, str, NoneType] = '5GB',
    create_pr: bool = False,
    safe_serialization: bool = True,
    revision: str = None,
    commit_description: str = None,
    tags: Optional[List[str]] = None,
    **deprecated_kwargs,
) -> str
Docstring:
Upload the model file to the 🤗 Model Hub.

Parameters:
    repo_id (`str`):
        The name of the repository you want to push your model to. It should contain your organization name
        when pushing to a given organization.
    use_temp_dir (`bool`, *optional*):
        Whether or not to use a temporary directory to store the files saved before they are pushed to the Hub.
        Will default to `True` if there is no directory named like `repo_id`, `False` otherwise.
    commit_message (`str`, *optional*):
   

# Tokenization

In [3]:
from models.model_utils import GPT2Model
import torch
from data.data_utils import create_instruct_dataset
from torch.utils.data import DataLoader

2025-09-13 22:28:15,495 - datasets - INFO - PyTorch version 2.5.1 available.


In [4]:
dataset = create_instruct_dataset(10, ["eng_spa"])
dataset

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 10
})

In [5]:
dataset[0]

{'prompt': 'Translate the following.\n\nEnglish: Tom was the one who cooked dinner.\n\nSpanish:',
 'completion': 'Tom fue el que cocinó la cena.'}

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wrapped_model = GPT2Model("openai-community/gpt2-medium", device)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [28]:
tokenizer_func = wrapped_model.tokenize_function
tokenized_dataset = dataset.map(
    tokenizer_func,
    remove_columns=["prompt", "completion"],
    batched=True,
    batch_size=5
)
tokenized_dataset

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'targets'],
    num_rows: 10
})

In [29]:
tokenized_dataset[0]

{'input_ids': [8291,
  17660,
  262,
  1708,
  13,
  198,
  198,
  15823,
  25,
  4186,
  373,
  262,
  530,
  508,
  15847,
  8073,
  13,
  198,
  198,
  43584,
  25,
  13787,
  37911,
  1288,
  8358,
  8954,
  259,
  10205,
  8591,
  269,
  8107,
  13,
  50256],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'targets': [13787, 37911, 1288, 8358, 8954, 259, 10205, 8591, 269, 8107, 13]}

In [23]:
def tokenize_function(example):
    #text = example["prompt"] + "\n" + example["completion"] -> OLD WAY
    text = [p + c + tokenizer.eos_token for p, c in zip(example["prompt"], example["completion"])]
    # join list of strings
    text = "".join(text)
    input_encodings = tokenizer(text, truncation=True) #Maybe dont use truncation
    target_encodings = tokenizer(example["completion"], truncation=True)

    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "targets": target_encodings["input_ids"]} #because its necesary

In [24]:
tokenize_function(dataset[0])

{'input_ids': [15751,
  50256,
  305,
  50256,
  321,
  50256,
  77,
  220,
  50256,
  28202,
  50256,
  2290,
  50256,
  3609,
  50256,
  83,
  220,
  50256,
  1453,
  50256,
  300,
  50256,
  83,
  220,
  50256,
  71,
  80,
  50256,
  12496,
  50256,
  304,
  50256,
  69,
  220,
  50256,
  420,
  50256,
  5439,
  50256,
  44601,
  50256,
  23013,
  50256,
  675,
  50256,
  72,
  10205,
  50256,
  77,
  220,
  50256,
  4743,
  50256,
  13,
  64,
  50256,
  198,
  220,
  50256,
  198,
  66,
  50256,
  36,
  68,
  50256,
  20471,
  50256,
  4908,
  50256,
  75,
  13,
  50256],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'targets': [13787, 37911, 1288, 8358,

In [ ]:
tokenizer_func(dataset[0])

{'input_ids': [[15751, 50256],
  [305, 50256],
  [321, 50256],
  [77, 220, 50256],
  [28202, 50256],
  [2290, 50256],
  [3609, 50256],
  [83, 220, 50256],
  [1453, 50256],
  [300, 50256],
  [83, 220, 50256],
  [71, 80, 50256],
  [12496, 50256],
  [304, 50256],
  [69, 220, 50256],
  [420, 50256],
  [5439, 50256],
  [44601, 50256],
  [23013, 50256],
  [675, 50256],
  [72, 10205, 50256],
  [77, 220, 50256],
  [4743, 50256],
  [13, 64, 50256],
  [198, 220, 50256],
  [198, 66, 50256],
  [36, 68, 50256],
  [20471, 50256],
  [4908, 50256],
  [75, 13, 50256]],
 'attention_mask': [[1, 1],
  [1, 1],
  [1, 1],
  [1, 1, 1],
  [1, 1],
  [1, 1],
  [1, 1],
  [1, 1, 1],
  [1, 1],
  [1, 1],
  [1, 1, 1],
  [1, 1, 1],
  [1, 1],
  [1, 1],
  [1, 1, 1],
  [1, 1],
  [1, 1],
  [1, 1],
  [1, 1],
  [1, 1],
  [1, 1, 1],
  [1, 1, 1],
  [1, 1],
  [1, 1, 1],
  [1, 1, 1],
  [1, 1, 1],
  [1, 1, 1],
  [1, 1],
  [1, 1],
  [1, 1, 1]],
 'targets': [13787, 37911, 1288, 8358, 8954, 259, 10205, 8591, 269, 8107, 13]}

In [30]:
tokenizer.decode(tokenized_dataset[0]["input_ids"])

'Translate the following.\n\nEnglish: Tom was the one who cooked dinner.\n\nSpanish:Tom fue el que cocinó la cena.<|endoftext|>'

In [32]:
tokenizer.decode(tokenize_function(dataset[0])["input_ids"])

'TT<|endoftext|>ro<|endoftext|>am<|endoftext|>n <|endoftext|>sf<|endoftext|>lu<|endoftext|>ae<|endoftext|>t <|endoftext|>ee<|endoftext|> l<|endoftext|>t <|endoftext|>hq<|endoftext|>eu<|endoftext|> e<|endoftext|>f <|endoftext|>oc<|endoftext|>lo<|endoftext|>lc<|endoftext|>oi<|endoftext|>wn<|endoftext|>ió<|endoftext|>n <|endoftext|>gl<|endoftext|>.a<|endoftext|>\n <|endoftext|>\nc<|endoftext|>Ee<|endoftext|>nn<|endoftext|>ga<|endoftext|>l.<|endoftext|>'

In [36]:
from models.model_utils import T5Model
wrapped_model2 = T5Model("google/flan-t5-base", device)
tokenizer_func2 = wrapped_model2.tokenize_function
tokenizer2 = wrapped_model2.get_tokenizer()

In [39]:
tokenizer_func2(dataset[0])

{'input_ids': [30355,
  15,
  8,
  826,
  5,
  1566,
  10,
  3059,
  47,
  8,
  80,
  113,
  8311,
  2634,
  5,
  5093,
  10,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [3059,
  7683,
  15,
  3,
  15,
  40,
  238,
  576,
  75,
  77,
  4922,
  50,
  197,
  29,
  9,
  5,
  1]}

In [41]:
tokenizer2.decode(tokenizer_func2(dataset[0])["input_ids"])

'Translate the following. English: Tom was the one who cooked dinner. Spanish:</s>'

In [42]:
tokenized_dataset2 = dataset.map(
    tokenizer_func2,
    remove_columns=["prompt", "completion"],
    batched=True,
    batch_size=500
)
tokenized_dataset2

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10
})

In [43]:
tokenized_dataset2[0]

{'input_ids': [30355,
  15,
  8,
  826,
  5,
  1566,
  10,
  3059,
  47,
  8,
  80,
  113,
  8311,
  2634,
  5,
  5093,
  10,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [3059,
  7683,
  15,
  3,
  15,
  40,
  238,
  576,
  75,
  77,
  4922,
  50,
  197,
  29,
  9,
  5,
  1]}